In [129]:
import json
import pandas as pd
import numpy as np

print("Pandas Version: " + pd.__version__)
print("Numpy Version: " + np.__version__)

Pandas Version: 1.1.2
Numpy Version: 1.19.2


In [130]:
FREQUENT_USER_ACTIVE_DAYS_MIN = 28
FREQUENT_USER_THRESHOLD = 4

In [131]:
df = pd.read_csv("../data/clean/trips.csv")

In [134]:
weather = pd.read_csv("../data/clean/weather.csv")

In [132]:
# Convert string timestamp into dataframe and add numerical values

df['date_from'] = pd.DatetimeIndex(df['date_from'])

df['from_year'] = pd.DatetimeIndex(df['date_from']).year
df['from_month'] = pd.DatetimeIndex(df['date_from']).month
df['from_day'] = pd.DatetimeIndex(df['date_from']).day
df['from_hour'] = pd.DatetimeIndex(df['date_from']).hour
df['from_minute'] = pd.DatetimeIndex(df['date_from']).minute
df['from_weekday'] = pd.DatetimeIndex(df['date_from']).weekday

In [133]:
# Convert string timestamp into dataframe and add numerical values

df['date_until'] = pd.DatetimeIndex(df['date_until'])

df['until_year'] = pd.DatetimeIndex(df['date_until']).year
df['until_month'] = pd.DatetimeIndex(df['date_until']).month
df['until_day'] = pd.DatetimeIndex(df['date_until']).day
df['until_hour'] = pd.DatetimeIndex(df['date_until']).hour
df['until_minute'] = pd.DatetimeIndex(df['date_until']).minute
df['until_weekday'] = pd.DatetimeIndex(df['date_until']).weekday

In [135]:
# Add 'datetime' as key in preparation for merging trip and weather data

df['datetime'] = pd.to_datetime(pd.DataFrame(
    {
        'year': df['from_year'],
        'month': df['from_month'],
        'day': df['from_day'],
        'hour': df['from_hour']
    }
))

weather['datetime'] = pd.to_datetime(pd.DataFrame(
    {
        'year': weather['year'],
        'month': weather['month'],
        'day': weather['day'],
        'hour': weather['hour']
    }
))

In [114]:
df.sample(3)

,bike_id,user_id,date_from,date_until,from_station_name,from_station_id,to_station_name,to_station_id,booked_via,duration_in_min,...,from_hour,from_minute,from_weekday,until_year,until_month,until_day,until_hour,until_minute,until_weekday,datetime
5895043,120358,C7E22F2A3FA4A72CC9C2C1FA1FE6E59D6CC2514F,2016-08-27 19:24:27,2016-08-27 19:33:21,Millerntorplatz/St.Pauli,131905,Neuer Pferdemarkt / Beim Grünen Jäger,131890,iPhone SRH,9,...,19,24,5,2016,8,27,19,33,5,2016-08-27 19:00:00
3368271,120315,8610566ACFB2B6C3D20AD8D2CEBE1172F352D636,2015-08-14 16:57:35,2015-08-14 17:20:46,Jungfernstieg / Ballindamm,131879,Gänsemarkt / Büschstraße,131884,iPhone CAB,24,...,16,57,4,2015,8,14,17,20,4,2015-08-14 16:00:00
5542993,120470,C732851B73E573AE3144537BD9FD1BC874D0D0ED,2016-07-21 13:26:57,2016-07-21 13:34:09,Harvestehuder Weg / Alte Rabenstraße,137994,Universität / Moorweidenstraße,131882,iPhone SRH,8,...,13,26,3,2016,7,21,13,34,3,2016-07-21 13:00:00


In [115]:
weather.sample(3)

,year,month,day,hour,precip_intensity,precip_probability,precip_type,precip_accumulation,temperature,apparent_temperature,dew_point,humidity,wind_speed,wind_gust,wind_bearing,uv_index,visibility,cloud_cover,pressure,datetime
20009,2016,4,13,18,0.0,0.0,none,0.003249,13.33,13.33,5.66,0.60,3.92,3.92,284.0,1.0,10.003,0.75,1006.00000,2016-04-13 18:00:00
29025,2017,4,24,10,0.0,0.0,none,0.004253,6.91,3.57,0.37,0.63,5.37,5.37,223.0,2.0,10.003,0.75,1013.57750,2017-04-24 10:00:00
26967,2017,1,28,15,0.0,0.0,none,0.158095,2.76,0.50,-4.89,0.57,2.24,2.24,155.0,0.0,10.003,0.00,1017.05625,2017-01-28 15:00:00


In [116]:
# Add weather data to every trip

trips = pd.merge(df, weather, on='datetime', left_index=False, right_index=False, how='left', sort=False)
trips.sample(5)

,bike_id,user_id,date_from,date_until,from_station_name,from_station_id,to_station_name,to_station_id,booked_via,duration_in_min,...,apparent_temperature,dew_point,humidity,wind_speed,wind_gust,wind_bearing,uv_index,visibility,cloud_cover,pressure
5518762,119986,58EA4C1E8C664A100A7F7B6C7898232E955820A4,2016-07-19 08:46:31,2016-07-19 09:01:24,Hofweg/Am Langenzug,200502,Schopenstehl/Alter Fischmarkt,218636,iPhone SRH,15,...,17.72,14.49,0.81,2.77,2.77,269.0,1.000,10.003,0.560,1018.215447
6371041,120545,97D3F1253CC8202704C3500DA98838464ED53BD2,2016-10-20 23:02:24,2016-10-20 23:25:01,Schulterblatt/Eifflerstraße,131648,Jarrestraße / Rambatzweg,138376,IVR,23,...,6.47,5.41,0.93,0.49,0.49,41.0,0.000,10.003,0.750,1014.142708
742117,143770,081C8710C397F6BC8565E61A6540A2C7E634F508,2014-05-30 17:57:49,2014-05-30 18:24:55,Hauptbahnhof Ost / Hachmannplatz,131873,Jarrestraße / Rambatzweg,138376,Terminal HH_34 (-2541-),28,...,18.77,7.20,0.47,4.63,4.63,293.0,4.375,9.988,0.265,1022.280000
1170450,143552,9598E64474F96A6A2971B721D9817395B7F89AF9,2014-07-27 07:47:45,2014-07-27 08:10:38,Bernhard-Nocht-Straße/Zirkusweg,196417,Mannesallee/Veringstraße,199402,Android SRH,23,...,18.16,16.64,0.92,1.29,1.29,114.0,0.000,10.003,0.750,1014.600000
1329890,120492,19C20CC796AC5CE2F0DDEDD1979D2E23AF1EC981,2014-08-15 22:01:10,2014-08-15 22:22:23,Bahnhof Dammtor Süd / Marseiller Straße,138382,Heußweg/Wiesenstraße,201326,iPhone SRH,22,...,15.61,11.53,0.77,0.97,0.97,292.0,0.000,10.003,0.190,1011.100000


In [136]:
# Select unique user IDs and identify first and last rental as well as number of entries per user

user_trip_dates = trips.groupby(['user_id'], as_index=False).agg({'date_from': [np.min,np.max]})
user_trip_dates.columns = pd.Index(['user_id', 'date_from_min', 'date_from_max'])

user_trip_count = trips['user_id'].value_counts()

user_df = pd.DataFrame({'user_id': user_trip_count.index, 'trips_count': user_trip_count.values})
user_df = user_trip_dates.merge(user_df)

user_df['active_period_in_days'] = user_df.apply(lambda x: (x['date_from_max'] - x['date_from_min']).days, axis=1)

In [137]:
# heuristic for determining if a user is 'casual' or 'frequent'

def is_frequent_user(trips_count, active_period_in_days):
    return (
        (active_period_in_days / trips_count) <= FREQUENT_USER_THRESHOLD
    ) & (
        active_period_in_days >= FREQUENT_USER_ACTIVE_DAYS_MIN
    )

user_df['is_frequent_user'] = user_df.apply(lambda x: is_frequent_user(x['trips_count'], x['active_period_in_days']), axis=1)

In [119]:
user_df[user_df.is_frequent_user == True].sample(5)

,user_id,date_from_min,date_from_max,trips_count,active_period_in_days,is_frequent_user
114661,81EE5A1C100D83A81DB10CFAB931C148F83962E8,2014-03-13 09:07:29,2017-05-12 12:43:49,302,1156,True
138871,9D863803D7041B2F32A52F649571DB8707CE6347,2017-01-04 06:30:38,2017-03-19 08:26:34,148,74,True
140122,9EF4753D4F94B4ECDB7641D0FC42D3EC6E500CC5,2015-05-15 13:23:57,2016-04-13 15:57:32,131,334,True
72127,516D2B2D15E55E8EBEE378A3C8D5D4A98D083AA4,2015-04-26 12:25:41,2015-08-30 20:59:54,142,126,True
160748,B66073FDB087A82D22DE310D93A6CF67063CA5BF,2014-01-07 10:54:37,2017-05-11 18:38:34,332,1220,True


In [120]:
user_df[user_df.is_frequent_user == False].sample(5)

,user_id,date_from_min,date_from_max,trips_count,active_period_in_days,is_frequent_user
142946,A22833C86AA521924AB1EC190080F59BA1334BEE,2015-06-06 14:16:13,2015-06-06 14:16:32,2,0,False
198323,E164BD7A9959CFD4DB4D2B6E1724F59D1037C2C6,2017-04-15 13:47:26,2017-04-17 17:11:27,11,2,False
113090,8011F6D4136180A07F75B1BE177C0FC029239AED,2014-04-29 21:41:55,2017-01-10 16:44:31,10,986,False
42909,306D8874A8A21E890F8AD8D4B3F6510ED32E6500,2014-07-27 13:36:23,2014-07-27 13:36:23,1,0,False
34553,2702BD212340FF9C2B2B6412DCC9ECA6C11BA30A,2015-05-20 15:25:13,2016-06-27 18:04:18,8,404,False


In [121]:
user_df.to_csv("../data/users.csv", index=False)

In [138]:
frequent_user_ids = user_df[user_df.is_frequent_user == True].user_id.unique()

frequent_user_trips = trips.loc[trips['user_id'].isin(frequent_user_ids)]
casual_user_trips = trips.loc[~trips['user_id'].isin(frequent_user_ids)]

In [125]:
# Add user type to combined trip data

frequent_user_mask = trips['user_id'].isin(frequent_user_ids)
trips['user_type'] = pd.Categorical(np.where(frequent_user_mask, 'frequent', 'casual'))

In [126]:
frequent_user_trips.to_csv("../data/frequent_user_trips.csv", index=False)

In [127]:
casual_user_trips.to_csv("../data/casual_user_trips.csv", index=False)

In [128]:
trips.to_csv("../data/all_trips.csv", index=False)

In [142]:
df[['datetime', 'from_station_name', 'to_station_name']].sample(5)

,datetime,from_station_name,to_station_name
6724916,2016-12-23 07:00:00,Kurt-Schumacher-Allee / Nagelsweg,Schopenstehl/Alter Fischmarkt
2124488,2015-01-28 07:00:00,Jarrestraße / Rambatzweg,Eduard-Rhein-Ufer / Schwanenwik
1989389,2014-12-08 17:00:00,Bahnhof Dammtor Süd / Marseiller Straße,Sievekingplatz/Gorch-Fock-Wall
3438391,2015-08-23 17:00:00,Neumühlen/Övelgönne,Bahnhof Altona West / Busbahnhof
6627317,2016-12-04 12:00:00,Lange Reihe / Kirchenallee,Goldbekplatz / Semperstraße


In [ ]:
# Count check-ins and check-outs per station per hour

df['from_datetime'] = pd.to_datetime(pd.DataFrame(
    {
        'year': df['from_year'],
        'month': df['from_month'],
        'day': df['from_day'],
        'hour': df['from_hour']
    }
))

df['until_datetime'] = pd.to_datetime(pd.DataFrame(
    {
        'year': df['until_year'],
        'month': df['until_month'],
        'day': df['until_day'],
        'hour': df['until_hour']
    }
))

In [237]:
check_ins = df[
    ['until_datetime', 'to_station_name']
].groupby(['until_datetime', 'to_station_name']).size().to_frame('check_ins')

check_outs = df[
    ['from_datetime', 'from_station_name']
].groupby(['from_datetime', 'from_station_name']).size().to_frame('check_outs')

check_ins.index.rename(['datetime', 'station_name'], inplace=True)
check_outs.index.rename(['datetime', 'station_name'], inplace=True)

station_count = check_ins.join(check_outs).fillna(0)
station_count.check_outs = station_count.check_outs.astype(int)
station_count.reset_index(inplace=True)
station_count.sample(10)

,datetime,station_name,check_ins,check_outs
529289,2014-10-29 05:00:00,Hauptbahnhof West / Glockengießer Wall,2,1
1702761,2016-06-28 17:00:00,Alter Postweg / Heimfelder Straße,1,0
1387177,2016-02-11 19:00:00,Eimsbütteler Straße/Waterloostraße,2,1
820437,2015-05-09 01:00:00,Jacobikirche / Steinstraße,1,0
1655299,2016-06-10 11:00:00,Hammerbrook / Sachsenfeld,1,2
1821777,2016-08-12 07:00:00,Auf dem Sande/Kehrwieder-MiniaturWunderland,4,1
1810940,2016-08-07 18:00:00,Wilhelmsburger Platz/Zur Schleuse,4,1
1871935,2016-08-30 20:00:00,Ottenser Marktplatz/Museumsstraße,3,7
404617,2014-08-23 10:00:00,Wandsbeker Marktstraße/Wandsbeker Marktplatz,1,0
1849740,2016-08-22 19:00:00,Bahrenfelder Straße/Völckersstraße,1,2


In [238]:
station_count = pd.merge(station_count, weather, on='datetime', left_index=False, right_index=False, how='left', sort=False)
station_count.sample(5)

,datetime,station_name,check_ins,check_outs,year,month,day,hour,precip_intensity,precip_probability,...,apparent_temperature,dew_point,humidity,wind_speed,wind_gust,wind_bearing,uv_index,visibility,cloud_cover,pressure
1952108,2016-09-28 20:00:00,Bahnhof Dammtor Süd / Marseiller Straße,4,6,2016,9,28,20,0.0000,0.0,...,17.51,11.73,0.69,2.52,3.44,271.0,0.0,10.003,0.620000,1015.552174
84296,2014-03-03 01:00:00,Goldbekplatz / Semperstraße,1,0,2014,3,3,1,0.0000,0.0,...,-0.29,-0.95,0.82,2.01,2.01,124.0,0.0,9.988,0.086364,1018.834483
749251,2015-03-30 20:00:00,Mannesallee/Veringstraße,1,0,2015,3,30,20,0.0000,0.0,...,1.71,1.17,0.71,7.39,14.45,273.0,0.0,10.003,0.750000,1002.600000
1972275,2016-10-06 21:00:00,U-Bahn Schlump,1,1,2016,10,6,21,0.0185,0.1,...,10.29,8.56,0.89,2.90,2.90,17.0,0.0,7.192,0.750000,1025.100000
1829737,2016-08-15 10:00:00,Weidestraße /Johannes-Prassek-Park,1,4,2016,8,15,10,0.0000,0.0,...,16.30,13.82,0.85,2.61,2.61,311.0,2.0,10.003,0.620000,1024.700000


In [239]:
station_count.to_csv("../data/station_count.csv", index=False)